In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [3]:
def eval_numerical_gradient_array(f, x, df, h=1e-5):
  """
  Evaluate a numeric gradient for a function that accepts a numpy
  array and returns a numpy array.
  """
  grad = np.zeros_like(x)
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:
    ix = it.multi_index
    
    oldval = x[ix]
    x[ix] = oldval + h
    pos = f(x)
    x[ix] = oldval - h
    neg = f(x)
    x[ix] = oldval
    
    grad[ix] = np.sum((pos - neg) * df) / (2 * h)
    it.iternext()
   
  return grad

In [4]:
def eval_numerical_gradient(f, x, verbose=False, h=0.00001):
  """ 
  a naive implementation of numerical gradient of f at x 
  - f should be a function that takes a single argument
  - x is the point (numpy array) to evaluate the gradient at
  """ 

  fx = f(x) # evaluate function value at original point
  grad = np.zeros_like(x)
  # iterate over all indexes in x
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # evaluate function at x+h
    ix = it.multi_index
    oldval = x[ix]
    x[ix] = oldval + h # increment by h
    fxph = f(x) # evalute f(x + h)
    x[ix] = oldval - h
    fxmh = f(x) # evaluate f(x - h)
    x[ix] = oldval # restore

    # compute the partial derivative with centered formula
    grad[ix] = (fxph - fxmh) / (2 * h) # the slope
    if verbose:
      print ix, grad[ix]
    it.iternext() # step to next dimension

  return grad

In [5]:
def affine_forward(x, w, b):
  """
  Computes the forward pass for an affine (fully-connected) layer.

  The input x has shape (N, d_1, ..., d_k) where x[i] is the ith input.
  We multiply this against a weight matrix of shape (D, M) where
  D = \prod_i d_i

  Inputs:
  x - Input data, of shape (N, d_1, ..., d_k)
  w - Weights, of shape (D, M)
  b - Biases, of shape (M,)
  
  Returns a tuple of:
  - out: output, of shape (N, M)
  - cache: (x, w, b)
  """
  out = None
  #############################################################################
  # TODO: Implement the affine forward pass. Store the result in out. You     #
  # will need to reshape the input into rows.                                 #
  #############################################################################
  N = x.shape[0]
  D = np.prod(x.shape[1:])
  x2 = np.reshape(x, (N, D))
  out = np.dot(x2, w) + b
  #############################################################################
  #                             END OF YOUR CODE                              #
  #############################################################################
  cache = (x, w, b)
  return out, cache

In [6]:
def affine_backward(dout, cache):
  """
  Computes the backward pass for an affine layer.

  Inputs:
  - dout: Upstream derivative, of shape (N, M)
  - cache: Tuple of:
    - x: Input data, of shape (N, d_1, ... d_k)
    - w: Weights, of shape (D, M)

  Returns a tuple of:
  - dx: Gradient with respect to x, of shape (N, d1, ..., d_k)
  - dw: Gradient with respect to w, of shape (D, M)
  - db: Gradient with respect to b, of shape (M,)
  """
  x, w, b = cache
  dx, dw, db = None, None, None
  #############################################################################
  # TODO: Implement the affine backward pass.                                 #
  #############################################################################
  N = x.shape[0]
  D = x.shape[1]
 

  dx = np.dot(dout, w.T) # N x D
  dw = np.dot(x.T, dout) # D x M
  db = np.dot(dout.T, np.ones(N)) # M x 1


  #############################################################################
  #                             END OF YOUR CODE                              #
  #############################################################################
  return dx, dw, db

In [7]:
x = np.random.randn(1, 3)
w = np.random.randn(3, 2)
b = np.random.randn(2)
dout = np.random.randn(1, 2)

dx_num = eval_numerical_gradient_array(lambda x: affine_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_forward(x, w, b)[0], b, dout)

_, cache = affine_forward(x, w, b)
dx, dw, db = affine_backward(dout, cache)

print dx_num, dx
# The error should be less than 1e-10
print 'Testing affine_backward function:'
print 'dx error: ', rel_error(dx_num, dx)
print 'dw error: ', rel_error(dw_num, dw)
print 'db error: ', rel_error(db_num, db)

[[ 1.15385272  1.51507982 -0.90811306]] [[ 1.15385272  1.51507982 -0.90811306]]
Testing affine_backward function:
dx error:  4.0564606137e-12
dw error:  4.81654210965e-12
db error:  5.00020125704e-13


In [8]:
def sigmoid_forward(z):
        sigmoidfn = 1.0 / (1.0 + np.exp(-z)) # sigmoid activation function
        return sigmoidfn
    
def sigmoid_backward(dout,fw_activation):
        return np.multiply(np.multiply(fw_activation,1-fw_activation),dout)

In [9]:
x = np.random.randn(1, 3)
w = np.random.randn(3, 2)
b = np.random.randn(2)
dout = np.random.randn(1, 2)

dx_num = eval_numerical_gradient_array(lambda x: sigmoid_forward(affine_forward(x, w, b)[0]), x, dout)
dw_num = eval_numerical_gradient_array(lambda w: sigmoid_forward(affine_forward(x, w, b)[0]), w, dout)
db_num = eval_numerical_gradient_array(lambda b: sigmoid_forward(affine_forward(x, w, b)[0]), b, dout)

y, cache = affine_forward(x, w, b)
sigout = sigmoid_forward(y)
douty = sigmoid_backward(dout,sigout)
dx, dw, db = affine_backward(douty, cache)

print dx_num, dx
# The error should be less than 1e-10
print 'Testing affine_backward function:'
print 'dx error: ', rel_error(dx_num, dx)
print 'dw error: ', rel_error(dw_num, dw)
print 'db error: ', rel_error(db_num, db)

[[ 0.12026964  0.1155564   0.19441144]] [[ 0.12026964  0.1155564   0.19441144]]
Testing affine_backward function:
dx error:  3.18948656806e-11
dw error:  1.61641298285e-11
db error:  1.17659911233e-11


In [10]:
def euclidean_sq_loss(z,t):
    #z is lower layer's activation, t is the target value
    m = z.shape[0] #number of training sample in a batch
    n = z.shape[1] #size of feature

    loss = 0.5* np.dot( z[0,:]-t[0,:],np.transpose(z[0,:]-t[0,:]) )
    dout = (z - t)
    
    return loss,dout

In [11]:
def piano_fw(x,w,b,t):
    loss,_ = euclidean_sq_loss(sigmoid_forward(affine_forward(x, w, b)[0]),t)
    return loss

In [12]:
x = np.random.randn(1, 3)
w = np.random.randn(3, 2)
b = np.random.randn(2)
t = np.array([[0,1]])

y, cache = affine_forward(x, w, b)
sigout = sigmoid_forward(y)
loss,dout = euclidean_sq_loss(sigout,t)
print loss, 0.5* (np.linalg.norm(sigout-t))**2

dw_num = eval_numerical_gradient(lambda w: piano_fw(x,w,b,t), w)
db_num = eval_numerical_gradient(lambda b: piano_fw(x,w,b,t), b)

print 'dw_num:',dw_num,'db_num:',db_num

douty = sigmoid_backward(dout,sigout)
dx, dw, db = affine_backward(douty, cache)
print dw,db
#print dx_num, dx
# The error should be less than 1e-10
#print 'Testing affine_backward function:'
#print 'dx error: ', rel_error(dx_num, dx)
print 'dw error: ', rel_error(dw_num, dw)
print 'db error: ', rel_error(db_num, db)

0.324826761628 0.324826761628
dw_num: [[-0.04460987  0.04581823]
 [ 0.08711245 -0.08947208]
 [ 0.29936559 -0.30747455]] db_num: [ 0.13772246 -0.14145297]
[[-0.04460987  0.04581823]
 [ 0.08711245 -0.08947208]
 [ 0.29936559 -0.30747455]] [ 0.13772246 -0.14145297]
dw error:  4.35688660668e-11
db error:  3.57901872442e-12


In [13]:
class NeuralNet_Piano:
    #class variables
    # num_classes  is the size of the last affine activation
    # input_dim   is the size of input dimension
    # num_hidden_layers is how many hidden layers
    ## (constructor ) Dict model
    ## (constructor) num_layers is how many totol layers = hidden_layers + 2(Softmax cross entropy loss layer)
    ## (Forward) Dict input cache
    ## (Backward) Dict gradient cache 
    #class def
    # constructor to construct the neural network, given the hyper parameter
    # Forward
    # Backward
    # applyGrad
    def __init__(self,model_hyper):
        num_classes = model_hyper['num_classes']
        input_dim  = model_hyper['input_dim']
        self.num_hidden_layers = model_hyper['num_hidden_layers']
        self.num_layers = self.num_hidden_layers + 1
        layer_list = [None] * self.num_hidden_layers
        cur_layer_size = input_dim
        self.learnrate = model_hyper['learnrate']
        self.NN_model = []
        self.NN_grad = []
        self.NN_num_grad = []
        for hid_layer_iter in range(self.num_hidden_layers):  
            layer_model = {}
            next_layer_size = model_hyper['layer_sizes'][hid_layer_iter]
            layer_model['W'] = 0.9 * np.random.randn(cur_layer_size, next_layer_size)
            layer_model['b'] =0.9 * np.random.randn(next_layer_size)
            self.NN_model.append(layer_model)
            cur_layer_size = next_layer_size
        
        layer_model = {}   
        next_layer_size = num_classes
        layer_model['W'] = 0.9 * np.random.randn(cur_layer_size, next_layer_size)
        self.NN_model.append(layer_model)
        self.NN_cache=[]
        self.sigmoid_on = False
        for i in self.NN_model:
            print i['W'].shape
            if 'b' in i:
                print i['b'].shape
            print "__________________"
            
    def affine_backward(self,dout, cache):
        x = cache['x']
        w = cache['w']
        b = cache['b']
        #print x,w,b
        grad_cache = {}
        dx, dw, db = None, None, None
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))

        dx2 = np.dot(dout, w.T) # N x D
        dw = np.dot(x2.T, dout) # D x M
        db = np.dot(dout.T, np.ones(N)) # M x 1

        dx = np.reshape(dx2, x.shape)
        grad_cache['dx'] = dx
        grad_cache['dw'] = dw
        #print grad_cache['dw']
        grad_cache['db'] = db
        return grad_cache
    
    def affine_backward2(self,dout, cache):
        x = cache['x']
        w = cache['w']

        grad_cache = {}
        dx, dw = None, None
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))

        dx2 = np.dot(dout, w.T) # N x D
        dw = np.dot(x2.T, dout) # D x M       

        dx = np.reshape(dx2, x.shape)
        grad_cache['dx'] = dx
        grad_cache['dw'] = dw
 
        return grad_cache  

    def affine_forward(self,x, w, b): 
        cache = {}
        out = None
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))
        out = np.dot(x2, w) + b
        cache['x'] = x
        cache['w'] = w
        cache['b'] = b
        return out, cache    
    
    def affine_forward2(self,x, w): 
        out = None
        cache = {}
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))
        out = np.dot(x2, w) 
        cache['x'] = x
        cache['w'] = w
        return out, cache  

    def sigmoid_forward(self,z):
        sigmoidfn = 1.0 / (1.0 + np.exp(-z)) # sigmoid activation function
        return sigmoidfn
    
    def sigmoid_backward(self,dout,fw_activation):
        return np.multiply(np.multiply(fw_activation,1-fw_activation),dout)

    def euclidean_sq_loss(self,z,t):
        #z is lower layer's activation, t is the target value
        m = z.shape[0] #number of training sample in a batch
        n = z.shape[1] #size of feature
        #print z.shape, t.shape
        loss = 0.5* np.dot( z[0,:]-t[0,:],np.transpose(z[0,:]-t[0,:]) )
        dout = (z - t)
    
        return loss,dout
    
    def forward_train(self,x,y):
        activation = x
        #print activation

        for i in self.NN_model:
            if 'b' in i:
                activation, cache = self.affine_forward(activation, i['W'], i['b'])
                self.NN_cache.append(cache)
            else:
                activation, cache = self.affine_forward2(activation, i['W'])
                self.NN_cache.append(cache)
        

        if (self.sigmoid_on == True):
            sigout = self.sigmoid_forward(activation) 
            self.NN_cache.append(sigout)
            loss,dz = self.euclidean_sq_loss(sigout,y)
        else:
            loss,dz = self.euclidean_sq_loss(activation,y)
        #for i in self.NN_cache:
        #    print i['w'].shape,i['x'].shape
        #    if 'b' in i:
        #        print i['b'].shape
        #    print "__________________"   
        return loss,dz
    
    def forward_test(self,x):
        activation = x
        #print activation

        for i in self.NN_model:
            if 'b' in i:
                activation, cache = self.affine_forward(activation, i['W'], i['b'])
                self.NN_cache.append(cache)
            else:
                activation, cache = self.affine_forward2(activation, i['W'])
                self.NN_cache.append(cache)
 
        return self.sigmoid_forward(activation)
    
    def backward_train(self,dz):
        if self.sigmoid_on == True:
            sigactivation = self.NN_cache[-1]
            backgrad = sigmoid_backward(dz,sigactivation)
            for i in reversed(self.NN_cache[:-1]):
                if 'b' in i:
                    grad_cache = self.affine_backward(backgrad,i)
                else:
                    grad_cache = self.affine_backward2(backgrad,i)
                backgrad = grad_cache['dx']
                
                self.NN_grad.insert(0,grad_cache)
        else:
            backgrad = dz
            for i in reversed(self.NN_cache):
                if 'b' in i:
                    grad_cache = self.affine_backward(backgrad,i)
                else:
                    grad_cache = self.affine_backward2(backgrad,i)
                backgrad = grad_cache['dx']
                
                self.NN_grad.insert(0,grad_cache)     

           
    def Apply_UpdateW(self):
        for i in range(self.num_layers):
            delta_W = -self.learnrate * self.NN_grad[i]['dw'] 
            #print delta_W
            self.NN_model[i]['W'] += delta_W
            #print 'Apply Update',i
            if 'db' in self.NN_grad[i] :
                delta_b = -self.learnrate * self.NN_grad[i]['db'] 
                self.NN_model[i]['b'] += delta_b
        self.reset_cache()

    def reset_cache(self):
        del(self.NN_cache[:])
        del(self.NN_grad[:]) 
        del(self.NN_num_grad[:]) 
        
    def get_grad(self):
        return self.NN_grad

    def get_W(self):
        return self.NN_model
    def cal_num_grad(self,w,h,x,y):
        grad = np.zeros_like(w)
        #print "w.shape",w.shape
        # iterate over all indexes in w
        it = np.nditer(w, flags=['multi_index'], op_flags=['readwrite'])
        
        while not it.finished:
            
            # evaluate function at x+h
            ix = it.multi_index
            oldval = w[ix]
            w[ix] = oldval + h # increment by h
            fxph,_ = self.forward_train(x,y) # evalute f(x + h)
            w[ix] = oldval - h
            fxmh,_ = self.forward_train(x,y) # evaluate f(x - h)
            w[ix] = oldval # restore
            # compute the partial derivative with centered formula
            grad[ix] = (fxph - fxmh) / (2 * h) # the slope
            it.iternext()           
        return grad
    def get_numerical_gradient(self):
        return self.NN_num_grad
    def set_numerical_gradient(self,x,y, h=0.0001):        
        for i in self.NN_model:
            num_grad_cache = {}
            num_grad_cache['dw'] = self.cal_num_grad(i['W'],h,x,y)
            
            if 'b' in i:
                num_grad_cache['db'] = self.cal_num_grad(i['b'],h,x,y)
               
            self.NN_num_grad.append(num_grad_cache)
            
        return self.NN_num_grad
    
    def check_grad_numgrad(self,x,y, h=0.0001,verbose = True):       
        self.set_numerical_gradient(x,y, h)  
        if verbose:
            for i in range(self.num_hidden_layers+1):
                print i,"th layer"
                if (rel_error(self.NN_grad[i]['dw'],self.NN_num_grad[i]['dw'])<1e-7):
                    print 'grad W correct',rel_error(self.NN_grad[i]['dw'],self.NN_num_grad[i]['dw'])
                else:
                    print self.NN_grad[i]['dw'].flatten()
                    print self.NN_num_grad[i]['dw'].flatten()
                    print rel_error(self.NN_grad[i]['dw'],self.NN_num_grad[i]['dw'])
                    print np.linalg.norm(self.NN_grad[i]['dw'].flatten() - self.NN_num_grad[i]['dw'].flatten())**2
                    print '==========='
                if 'db' in self.NN_grad[i] :
                    if (rel_error(self.NN_grad[i]['db'],self.NN_num_grad[i]['db'])<1e-7):
                        print 'grad b correct',rel_error(self.NN_grad[i]['db'],self.NN_num_grad[i]['db'])
                    else:
                        print self.NN_grad[i]['db'].flatten()
                        print self.NN_num_grad[i]['db'].flatten()
                        print rel_error(self.NN_grad[i]['db'],self.NN_num_grad[i]['db'])
                        print '==========='

In [14]:
model_hyper = {}
model_hyper['num_classes'] = 4
model_hyper['input_dim']=4
model_hyper['num_hidden_layers']=0
model_hyper['learnrate']=0.0001
layer_size = [None] * model_hyper['num_hidden_layers']
#layer_size[0] = 4
#layer_size[1] = 4
#layer_size[2] = 5
#layer_size[3] = 6
model_hyper['layer_sizes'] = layer_size  

In [15]:
a_nn = NeuralNet_Piano(model_hyper)
w_org = a_nn.get_W()
w_org0 = w_org[0]['W'].copy()
num_inputs = 1
x = np.random.randn(num_inputs, model_hyper['input_dim'])
y = np.random.randint(model_hyper['num_classes'] , size=num_inputs)
y_onehot=np.zeros([num_inputs,model_hyper['num_classes']])
y_onehot[np.arange(num_inputs), y] = 1
y = y_onehot.astype(int)
print 'xshape',x.shape,y.shape
print y,'\n',y_onehot
print '----------'
loss,dz = a_nn.forward_train(x,y)
print 'dz shape',dz.shape
a_nn.backward_train(dz)
grad = a_nn.get_grad()
a_nn.check_grad_numgrad(x,y)
print y.dtype,x.dtype

(4, 4)
__________________
xshape (1, 4) (1, 4)
[[0 1 0 0]] 
[[ 0.  1.  0.  0.]]
----------
dz shape (1, 4)
0 th layer
grad W correct 2.32408300462e-12
int64 float64


In [16]:
w_after = a_nn.get_W()
print w_org[0]['W'] - w_after[0]['W']
grad_w = a_nn.get_grad()
delta_w = -model_hyper['learnrate'] * grad_w [0]['dw'] 
print delta_w

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[ -2.48222373e-05   1.08611905e-05  -5.19394212e-05   4.58539015e-05]
 [  4.55488021e-05  -1.99302831e-05   9.53088310e-05  -8.41419031e-05]
 [  2.82509230e-05  -1.23614424e-05   5.91137928e-05  -5.21876826e-05]
 [  1.14877180e-04  -5.02655309e-05   2.40375361e-04  -2.12211608e-04]]


In [17]:
num_grad = a_nn.get_numerical_gradient()

In [18]:
for i in a_nn.get_W():
    print i['W'].shape
for i in grad:
    print i['dw'].shape
for i in num_grad:
    print i['dw'].shape

(4, 4)
(4, 4)
(4, 4)


In [19]:
print '=====num grad ====='
for i in range(model_hyper['num_hidden_layers']+1):
    print i,"th layer:W error",rel_error(grad[i]['dw'],num_grad[i]['dw'])    
    if 'db' in grad[i] :
        print i,"th layer:b error",rel_error(grad[i]['db'],num_grad[i]['db'])
    print "__________________"  

print grad[0]['dw'].flatten()
print num_grad[0]['dw'].flatten()
print "__________________"  


=====num grad =====
0 th layer:W error 2.32408300462e-12
__________________
[ 0.24822237 -0.1086119   0.51939421 -0.45853902 -0.45548802  0.19930283
 -0.95308831  0.84141903 -0.28250923  0.12361442 -0.59113793  0.52187683
 -1.1487718   0.50265531 -2.40375361  2.12211608]
[ 0.24822237 -0.1086119   0.51939421 -0.45853902 -0.45548802  0.19930283
 -0.95308831  0.84141903 -0.28250923  0.12361442 -0.59113793  0.52187683
 -1.1487718   0.50265531 -2.40375361  2.12211608]
__________________


In [20]:
a_nn.Apply_UpdateW()
print y.dtype,x.dtype
loss,dz = a_nn.forward_train(x,y)
print 'dz shape',dz.shape
a_nn.backward_train(dz)
grad = a_nn.get_grad()
a_nn.check_grad_numgrad(x,y)
num_grad = a_nn.get_numerical_gradient()
print '=====num grad ====='
for i in range(model_hyper['num_hidden_layers']+1):
    print i,"th layer:W error",rel_error(grad[i]['dw'],num_grad[i]['dw'])    
    if 'db' in grad[i] :
        print i,"th layer:b error",rel_error(grad[i]['db'],num_grad[i]['db'])
    print "__________________"  

print grad[0]['dw'].flatten()
print num_grad[0]['dw'].flatten()
print "__________________"  


int64 float64
dz shape (1, 4)
0 th layer
grad W correct 7.53467841843e-12
=====num grad =====
0 th layer:W error 7.53467841843e-12
__________________
[ 0.24811399 -0.10856448  0.51916742 -0.4583388  -0.45528913  0.19921581
 -0.95267215  0.84105163 -0.28238587  0.12356045 -0.59087981  0.52164895
 -1.14827019  0.50243583 -2.40270402  2.12118947]
[ 0.24811399 -0.10856448  0.51916742 -0.4583388  -0.45528913  0.19921581
 -0.95267215  0.84105163 -0.28238587  0.12356045 -0.59087981  0.52164895
 -1.14827019  0.50243583 -2.40270402  2.12118947]
__________________


In [21]:
w_after = a_nn.get_W()
w_after0= w_after[0]['W']
real_delta_w = w_org0 - w_after0

print 'w_after0:\n',w_after0
print 'w_org0:\n',w_org0
print 'real_delta_w:\n',real_delta_w
grad_w = a_nn.get_grad()
delta_w = model_hyper['learnrate'] * grad_w [0]['dw'] 
print 'delta_w:\n',delta_w

w_after0:
[[ 1.0798828  -0.16304117  0.26107005 -1.25068364]
 [-1.04623898  0.59383889 -0.63021341 -0.2434686 ]
 [ 1.04877368  0.79858069 -1.85095499  0.98569323]
 [ 0.05775116 -0.85970158  0.06573663  0.19811627]]
w_org0:
[[ 1.07990763 -0.16305203  0.26112199 -1.25072949]
 [-1.04628453  0.59385882 -0.63030872 -0.24338446]
 [ 1.04874543  0.79859305 -1.85101411  0.98574542]
 [ 0.05763628 -0.85965131  0.06549626  0.19832848]]
real_delta_w:
[[  2.48222373e-05  -1.08611905e-05   5.19394212e-05  -4.58539015e-05]
 [ -4.55488021e-05   1.99302831e-05  -9.53088310e-05   8.41419031e-05]
 [ -2.82509230e-05   1.23614424e-05  -5.91137928e-05   5.21876826e-05]
 [ -1.14877180e-04   5.02655309e-05  -2.40375361e-04   2.12211608e-04]]
delta_w:
[[  2.48113988e-05  -1.08564480e-05   5.19167422e-05  -4.58338797e-05]
 [ -4.55289135e-05   1.99215807e-05  -9.52672151e-05   8.41051631e-05]
 [ -2.82385874e-05   1.23560449e-05  -5.90879811e-05   5.21648952e-05]
 [ -1.14827019e-04   5.02435828e-05  -2.40270402e-0

In [22]:
w_org0 = w_after0.copy()
a_nn.Apply_UpdateW()
print y.dtype,x.dtype
loss,dz = a_nn.forward_train(x,y)
print 'dz shape',dz.shape
a_nn.backward_train(dz)
grad = a_nn.get_grad()
a_nn.check_grad_numgrad(x,y_onehot)
num_grad = a_nn.get_numerical_gradient()
print '=====num grad ====='
for i in range(model_hyper['num_hidden_layers']+1):
    print i,"th layer:W error",rel_error(grad[i]['dw'],num_grad[i]['dw'])    
    if 'db' in grad[i] :
        print i,"th layer:b error",rel_error(grad[i]['db'],num_grad[i]['db'])
    print "__________________"  

print grad[0]['dw'].flatten()
print num_grad[0]['dw'].flatten()
print "__________________"  


int64 float64
dz shape (1, 4)
0 th layer
grad W correct 2.93664388561e-12
=====num grad =====
0 th layer:W error 2.93664388561e-12
__________________
[ 0.24800565 -0.10851708  0.51894073 -0.45813867 -0.45509034  0.19912882
 -0.95225617  0.84068439 -0.28226257  0.1235065  -0.59062181  0.52142118
 -1.14776881  0.50221644 -2.4016549   2.12026327]
[ 0.24800565 -0.10851708  0.51894073 -0.45813867 -0.45509034  0.19912882
 -0.95225617  0.84068439 -0.28226257  0.1235065  -0.59062181  0.52142118
 -1.14776881  0.50221644 -2.4016549   2.12026327]
__________________


In [23]:
w_after = a_nn.get_W()
w_after0= w_after[0]['W']
real_delta_w = w_org0 - w_after0

print 'w_after0:\n',w_after0
print 'w_org0:\n',w_org0
print 'real_delta_w:\n',real_delta_w
grad_w = a_nn.get_grad()
delta_w = model_hyper['learnrate'] * grad_w [0]['dw'] 
print 'delta_w:\n',delta_w

w_after0:
[[ 1.07985799 -0.16303032  0.26101813 -1.2506378 ]
 [-1.04619345  0.59381896 -0.63011814 -0.24355271]
 [ 1.04880191  0.79856833 -1.85089591  0.98564107]
 [ 0.05786598 -0.85975182  0.0659769   0.19790415]]
w_org0:
[[ 1.0798828  -0.16304117  0.26107005 -1.25068364]
 [-1.04623898  0.59383889 -0.63021341 -0.2434686 ]
 [ 1.04877368  0.79858069 -1.85095499  0.98569323]
 [ 0.05775116 -0.85970158  0.06573663  0.19811627]]
real_delta_w:
[[  2.48113988e-05  -1.08564480e-05   5.19167422e-05  -4.58338797e-05]
 [ -4.55289135e-05   1.99215807e-05  -9.52672151e-05   8.41051631e-05]
 [ -2.82385874e-05   1.23560449e-05  -5.90879811e-05   5.21648952e-05]
 [ -1.14827019e-04   5.02435828e-05  -2.40270402e-04   2.12118947e-04]]
delta_w:
[[  2.48005651e-05  -1.08517076e-05   5.18940731e-05  -4.58138666e-05]
 [ -4.55090336e-05   1.99128821e-05  -9.52256173e-05   8.40684392e-05]
 [ -2.82262572e-05   1.23506497e-05  -5.90621808e-05   5.21421178e-05]
 [ -1.14776881e-04   5.02216442e-05  -2.40165490e-0

In [26]:
num_train = 50000
y = np.random.randint(model_hyper['num_classes'] , size=num_train)
y_onehot=np.zeros([num_train,model_hyper['num_classes']])

y_onehot[np.arange(num_train), y] = 1
x = y_onehot
y = y_onehot.astype(int)
model2_hyper = {}
model2_hyper['num_classes'] = 4
model2_hyper['input_dim']=4
model2_hyper['num_hidden_layers']=1
model2_hyper['learnrate']=0.001
layer_size = [None] * model2_hyper['num_hidden_layers']
layer_size[0] = 4
#layer_size[1] = 4
#layer_size[2] = 5
#layer_size[3] = 6
model2_hyper['layer_sizes'] = layer_size  

num_round = 2
batch_size =1
num_train_batch = num_train/batch_size
num_rec = num_train
cost_rec = np.zeros(num_rec*num_round,dtype=np.float)
a_nn2 = NeuralNet_Piano(model2_hyper)

for ii in range(num_round):
    for i in range(num_rec):
        th_batch = i%num_train_batch
        Xbatch = x[th_batch*batch_size:th_batch*batch_size+batch_size,:]
        Ybatch = y[th_batch*batch_size:th_batch*batch_size+batch_size].astype(int)  
        #print Xbatch
        loss,dz = a_nn2.forward_train(Xbatch,Ybatch)
        print loss
        #if i < num_rec:
        cost_rec[ii*num_train+i] = loss
        a_nn2.backward_train(dz)
        #iris_nn.check_grad_numgrad(Xbatch,Ybatch,0.0001,True)
        a_nn2.Apply_UpdateW()

(4, 4)
(4,)
__________________
(4, 4)
__________________
22.7019236721
21.5643777453
20.4072390698
9.05056255148
12.2577177505
19.6553909984
18.6752842593
17.8165237618
16.9091206012
16.8526617304
9.16397731735
15.2687509663
14.5224070144
8.54188919353
14.8656698343
8.17480862631
7.45520621597
13.8208339862
7.77403129119
6.66459572217
6.29383049305
7.36947253287
12.5880519061
7.07624158875
5.5945618941
12.1664892745
6.84339928274
5.0272188281
11.26419146
4.52558057187
10.5492918906
4.07754684345
6.538733161
10.6803480662
9.69927755903
6.38452004088
9.3139754625
8.914684143
3.15443798199
2.99130041534
9.39087963632
8.9766414007
8.58549035799
2.47911855245
8.11144086909
7.4603071294
7.15814423632
2.04792691296
6.58643683592
7.49497145339
1.85459641328
6.59690109068
6.40907912441
7.07294144746
1.59735195187
1.52141483855
6.17783074563
1.43866873083
5.88542254677
1.36309481865
1.29733929887
5.60673134881
6.72327262977
5.92386678916
6.33563185008
5.57246370647
6.16151111232
5.39736627491
0.

0.200213996504
0.151378118679
0.194360133788
0.361023537135
0.147422791514
0.193602058435
0.194374343832
0.190860499634
0.187409506619
0.361248690025
0.193325100773
0.152775602609
0.187053833778
0.361899786607
0.153319835275
0.357897829099
0.196199966369
0.355950034515
0.349122989364
0.342432359504
0.18589521291
0.20618812188
0.182260232325
0.340581889112
0.178854757176
0.333975528387
0.327589311687
0.176714726052
0.178921714502
0.324506086116
0.176589980211
0.169839226771
0.215650255181
0.329807203907
0.165041528458
0.214088233987
0.157273487332
0.207735418169
0.202934677249
0.14869248351
0.347752030626
0.341105961623
0.149180683936
0.205523234853
0.200781226486
0.347513872896
0.340880261888
0.147205981174
0.204590945241
0.199874926285
0.139149203139
0.133843766704
0.193073196251
0.127733843877
0.187680127979
0.204981735433
0.18306778153
0.178869197124
0.123627693895
0.118928578379
0.173337737888
0.113765979767
0.16877305122
0.108893565
0.104761016825
0.100786243702
0.219697126564
0.3

0.348295475148
0.160839135343
0.154849873963
0.114169455226
0.112042379193
0.34676652005
0.156954482969
0.136179493233
0.151225306073
0.133323467021
0.145774818963
0.140348449706
0.120127562055
0.130324795529
0.117388494198
0.12695905564
0.364036870253
0.357490291785
0.351061227052
0.1349487086
0.114199872803
0.150821539213
0.114855232357
0.112714575872
0.150748716699
0.130882222122
0.112910295027
0.148159519878
0.113598410258
0.145420981219
0.127620611683
0.124806676893
0.11330634822
0.367911734988
0.145580821223
0.140162112138
0.134945699124
0.367671621288
0.119402204064
0.13481505644
0.129550939073
0.366674409514
0.119316922198
0.117092484993
0.129193095554
0.363478053222
0.14032020461
0.117116634653
0.359086088197
0.133259935294
0.140331566787
0.135109211532
0.130081766271
0.362851763328
0.134815165985
0.127785504926
0.123031440607
0.3643075145
0.127208992237
0.124837704101
0.357507953316
0.138457073298
0.121856982757
0.354544639061
0.119513223931
0.117283915557
0.115095739702
0.13

0.180056739454
0.176137347341
0.280807758145
0.101217787448
0.275773887916
0.0993194619997
0.0974108703513
0.177799337335
0.0949634532482
0.274659003435
0.0932183087637
0.0914258452727
0.269890198611
0.158613169899
0.179304432866
0.152459295998
0.146815494778
0.276531897183
0.0961749008775
0.1780416119
0.093757015707
0.148353565522
0.173440595477
0.281515211393
0.0937576314343
0.172761958034
0.0914098530996
0.08965146161
0.167943897392
0.164290817585
0.086906282665
0.160223331368
0.156739241247
0.0842744181434
0.295001408891
0.156517440089
0.153114817961
0.149786601448
0.0813561687578
0.146095197732
0.0793622952548
0.304389893374
0.0779195665944
0.145712927986
0.168751886158
0.14267678531
0.162663276424
0.139776641749
0.156868322465
0.151055946969
0.31785985188
0.312124646296
0.306492232146
0.0850887550077
0.301010067393
0.295576721364
0.290240736988
0.28500036432
0.0838125031054
0.165571333118
0.0845709580722
0.161802167355
0.155810025746
0.0876915463571
0.160383942865
0.152399211497


0.0728613392172
0.153958547641
0.140586574066
0.072999350011
0.137457073408
0.1503771442
0.285451562731
0.150555108495
0.0727049207713
0.136904107102
0.147013810941
0.0728727523355
0.289280815467
0.146793588054
0.287403747979
0.0710764829459
0.0696657074171
0.282307457986
0.0683752350024
0.146862514653
0.0690855659311
0.148748133562
0.14363359418
0.138331036054
0.133224808472
0.0735104197051
0.145673080863
0.071587540638
0.132731331023
0.0722614209076
0.141931402646
0.138881826486
0.13061528579
0.125793187229
0.305209434831
0.074122859863
0.12518100697
0.139942575274
0.136935473012
0.133993323479
0.131114706824
0.122366645986
0.128817323838
0.0746764179902
0.125625730456
0.121099535535
0.12352113068
0.0745164602553
0.120457353661
0.330818044301
0.324772987707
0.124304464572
0.123821526678
0.119249732865
0.114847172403
0.12340093891
0.33022470743
0.12392665973
0.121265233682
0.33055577202
0.121845155569
0.0765819893323
0.327616214406
0.0749818550752
0.0734933542937
0.321503026885
0.3156

...

1.53414913805e-06
1.00811877556e-09
1.23194251799e-05
1.36088923425e-05
1.53874523251e-06
1.49635139824e-06
1.45512555637e-06
1.2508251212e-05
1.35021200598e-05
1.46204914744e-06
1.42176840747e-06
1.25836386889e-05
1.21477446086e-05
1.17269506158e-05
1.69978396949e-06
1.6463714927e-08
1.60923396226e-08
1.57293415762e-08
1.39686313491e-05
1.58538508074e-06
6.60529763641e-09
1.54042155769e-06
1.19767461714e-05
1.60340060833e-06
1.1480054418e-08
1.16766082055e-05
1.6635708695e-06
1.61773762187e-06
1.14835361428e-05
1.67887183481e-06
1.42446085125e-05
1.14894231241e-05
1.42949577914e-05
1.61429349596e-06
1.14936509088e-05
2.98952399125e-08
1.42707803913e-05
1.14071948843e-05
1.43198692133e-05
1.13083317127e-05
1.76095886793e-06
1.10241486875e-05
4.77706522779e-08
1.45826111713e-05
1.09546433811e-05
1.46234174649e-05
1.43683811519e-05
1.4117792715e-05
1.38715745858e-05
1.17570652519e-05
1.52253980947e-08
1.48819832495e-08
1.39108786346e-05
7.41900167296e-09
1.16744622334e-05
1.39600525049e-

3.77399796071e-08
1.29066193813e-05
2.49789005863e-08
1.7296618969e-06
1.26106571143e-05
1.23907369354e-05
1.21746519714e-05
1.07918707843e-05
1.04181111126e-05
1.2496097173e-05
1.65326788463e-06
1.22202030095e-05
1.06892514282e-05
9.49394766519e-09
1.65070497573e-06
1.04281714739e-05
1.24756937781e-05
1.22581284561e-05
1.20443572623e-05
1.08632504354e-05
5.18844160837e-09
5.07144906209e-09
1.20923910183e-05
1.5770747761e-06
1.03963605586e-09
1.53318672614e-06
1.09676011112e-05
1.59151266123e-06
1.19807540815e-05
1.49268923064e-06
1.1054432566e-05
1.19861465444e-05
1.09382962112e-05
4.30139699982e-09
1.59571074584e-06
1.06653828405e-05
1.22505173504e-05
1.20368794315e-05
1.20492307068e-09
1.53896910259e-06
7.68836356193e-10
1.09311302122e-05
1.20361526159e-05
1.18262528886e-05
1.10850645192e-05
1.18875360142e-05
1.09671274432e-05
1.1947000559e-05
1.5773118597e-06
1.0953981436e-05
1.63577103436e-06
1.18930471744e-05
1.5351652245e-06
1.10428292014e-05
2.23826921339e-09
1.59312646578e-06


9.89541968191e-06
9.55276163458e-06
2.5983754397e-09
1.08821024434e-05
1.0692341355e-05
1.51492020593e-06
1.04545506577e-05
3.30187891934e-09
1.00273963927e-05
9.68016834324e-06
9.34496467099e-06
1.70943473937e-06
3.57646482414e-09
3.49582564491e-09
3.41700464972e-09
3.33996084374e-09
1.09190399227e-05
9.37056870549e-06
1.09649229544e-05
1.6451969679e-06
1.59987049412e-06
1.55579279943e-06
1.51292947943e-06
3.25096376325e-10
3.17618657138e-10
1.47133477624e-06
9.75126612447e-06
1.07505113157e-05
1.47192950814e-06
1.05123005147e-05
1.38183336284e-06
1.00729796997e-05
3.81611903617e-10
1.05207654879e-05
1.03373046507e-05
5.15220992483e-09
1.33849343481e-06
1.01138052075e-05
9.93744061409e-06
1.21019427958e-06
2.10509561993e-08
2.05745249991e-08
1.18097604925e-06
2.21892829024e-08
2.1687138352e-08
2.11963580806e-08
1.08826990807e-05
9.96186132635e-06
1.73979486923e-08
1.0736033977e-05
1.00377768962e-05
1.41105104184e-08
1.0594917656e-05
5.84904073265e-09
5.71644064349e-09
1.02159116107e-0

6.83424917021e-10
9.45069222018e-06
2.83529074513e-09
9.64331765615e-06
9.30943636304e-06
1.01589002189e-06
6.15003974031e-10
9.06280013036e-06
8.74901922568e-06
1.01325424044e-05
1.09469876647e-06
9.91336128395e-06
1.07549064347e-09
9.73872493163e-06
9.82423275239e-07
9.55356742277e-07
9.29035916158e-07
9.45237047598e-06
9.62280747496e-06
9.51044317534e-06
1.6733239479e-09
1.63530857991e-09
9.34907279394e-06
9.18604906333e-06
9.95025625534e-06
9.60574737562e-06
4.36057484973e-10
9.47251628416e-06
9.3468947302e-07
9.26937158806e-06
6.61713843717e-09
9.11234666119e-06
1.00040073717e-05
9.09222527624e-07
9.73025748613e-06
9.3618265791e-06
9.19185234824e-07
5.02054432251e-09
8.94731639956e-07
5.80699429195e-09
9.13260300534e-06
8.35043137601e-07
8.1203720696e-07
1.01109926295e-05
8.60295306088e-07
8.36593629195e-07
8.13544951399e-07
7.91131282152e-07
8.98629167669e-06
1.02644486564e-05
8.04585676894e-07
9.97824928447e-06
9.24311040727e-06
9.8552018962e-06
8.87335058606e-07
9.26803824545e-

7.35955246915e-06
2.22114956654e-08
2.1710242611e-08
9.46014870084e-06
1.10595975097e-06
9.25391960844e-06
1.03521402364e-06
1.00669286379e-06
7.72019754758e-06
7.45293983319e-06
1.89266772513e-08
1.11489008505e-06
1.66527704394e-08
1.62770025336e-08
1.08065110834e-06
1.05087802624e-06
7.4284362241e-06
2.16654122972e-08
2.11764776825e-08
7.19126039833e-06
3.17020892934e-08
1.15330550466e-06
7.02373041726e-06
6.78058471567e-06
9.96437876386e-06
6.73651714875e-06
5.71831011489e-08
6.51889747649e-06
1.01573851872e-05
9.9802771396e-06
4.35691912399e-08
6.68567241221e-06
1.3309493484e-06
6.5261184796e-06
6.30019967522e-06
1.43905069846e-06
1.02656309494e-05
1.00866359133e-05
9.91076184544e-06
9.73795433103e-06
2.8341358443e-08
9.55828842886e-06
1.8810868436e-08
1.83863949456e-08
1.79714999596e-08
9.36804749045e-06
9.20470235934e-06
5.64447287494e-09
7.53510502921e-06
1.14819566288e-06
1.11656179889e-06
9.02103003446e-09
7.42187832509e-06
1.65050390623e-08
1.15318425087e-06
1.12141288988e-06

In [30]:
Wmodel = a_nn2.get_W()
print Wmodel[0]['W']
print len(Wmodel)

print Wmodel[0]['b']
print len(Wmodel)

print Wmodel[1]['W']
print len(Wmodel)



[[ 2.05008239 -0.3145461  -0.23187051  0.65304911]
 [ 1.41127909 -0.40373397 -0.86683345  0.21370163]
 [-0.30752537  0.29390434  0.97623555 -1.47032883]
 [ 1.96765237  0.25319656 -0.22397309 -1.0347597 ]]
2
[ 1.09581567  0.13752367  0.68319132 -1.73064308]
2
[[ 0.36082029 -0.23841925 -0.17796907  0.19946775]
 [-0.17044022 -1.92495064 -0.69193391  1.9506015 ]
 [ 0.87742816 -0.62013535  0.43526799 -0.50557847]
 [ 0.52088307 -0.63952    -0.22357975  0.05016058]]
2


In [ ]:
import matplotlib.pyplot as plt
plt.plot(cost_rec[0::100])
plt.show()


DataXy_ = np.loadtxt(open("iris_prep.txt", "rb"), delimiter=",", skiprows=1)
print DataXy_.shape
rand_idx = np.random.permutation(149)
DataXy =  DataXy_[rand_idx,:]
print DataXy[0:4,:]
num_train = 100
num_test = 49
trainX = DataXy[0:num_train,:-1]
meanX = trainX.mean()
#print trainX
trainX = 0.1*(trainX-meanX) 
trainY = DataXy[0:num_train,-1]
trainY_onehot = np.zeros([num_train,3])
trainY_onehot[np.arange(num_train), trainY.astype(int)] = 1
trainY = trainY_onehot
testX = 0.1*(DataXy[num_train:,:-1]-meanX) 
testY = DataXy[num_train:,-1]
testY_onehot = np.zeros([num_test,3])
testY_onehot[np.arange(num_test), testY.astype(int)] = 1
testY = testY_onehot
#print trainX

model_hyper2 = {}
model_hyper2['num_classes'] = 3
model_hyper2['input_dim']=4
model_hyper2['num_hidden_layers']=3
model_hyper2['learnrate']=0.0001
layer_size = [None] * model_hyper2['num_hidden_layers']
layer_size[0] = 5
layer_size[1] = 4
layer_size[2] = 5
#layer_size[3] = 6
model_hyper2['layer_sizes'] = layer_size    

batch_size =1
num_train_batch = num_train/batch_size
num_rec = num_train
cost_rec = np.zeros(num_rec,dtype=np.float)
iris_nn = NeuralNet_Piano(model_hyper2)
#iris_nn.reset_cache()
for i in range(50000):
    th_batch = i%num_train_batch
    Xbatch = trainX[th_batch*batch_size:th_batch*batch_size+batch_size,:]
    #print Xbatch
    Ybatch = trainY[th_batch*batch_size:th_batch*batch_size+batch_size].astype(int)  
    #print Ybatch
    loss_iris,dz_iris = iris_nn.forward_train(Xbatch,Ybatch)
    if i < num_rec:
        cost_rec[i] = loss_iris
    print loss_iris
    iris_nn.backward_train(dz_iris)
    #iris_nn.check_grad_numgrad(Xbatch,Ybatch,0.0001,True)
    iris_nn.Apply_UpdateW()

import matplotlib.pyplot as plt
plt.plot(cost_rec[0:100])
plt.show()